<a href="https://colab.research.google.com/github/geraldgong/Kaggle_shopee/blob/gong/ArgFace_EfficientNetB4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon May 10 09:41:54 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Initialization

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Get Kaggle Data

In [50]:
!pip install -q kaggle
from google.colab import files
files.upload()

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [52]:
!kaggle datasets download -d ragnar123/shopee-tf-records-512-stratified
!unzip -q ./shopee-tf-records-512-stratified.zip

In [3]:
# Copy from Google Drive and unzip
!cp ./drive/MyDrive/data/shopee-product-matching.zip ./
!unzip -q shopee-product-matching.zip
!rm shopee-product-matching.zip

!cp ./drive/MyDrive/data/universal-sentence-encoder-cmlm_multilingual-preprocess_2.tar.gz ./
!cp ./drive/MyDrive/data/2.tar.gz ./
!mkdir preprocessor
!mkdir labse_model
!tar -xf universal-sentence-encoder-cmlm_multilingual-preprocess_2.tar.gz -C ./preprocessor
!tar -xf 2.tar.gz -C ./labse_model
!rm universal-sentence-encoder-cmlm_multilingual-preprocess_2.tar.gz 2.tar.gz

## Import Packages

In [60]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split

import os
import gc
import pandas as pd
import numpy as np
import cv2
import random
import math
import re
from tqdm import tqdm

In [5]:
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs:", len(physical_devices))

strategy = tf.distribute.get_strategy()

Num GPUs: 1


# Process Data

## Prepare .tfrec files

In [8]:
def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_example(feature0, feature1, feature2, feature3, feature4, feature5):
    feature = {
        'posting_id': _bytes_feature(feature0.encode('utf-8')),
        'image': _bytes_feature(feature1),
        'image_name': _bytes_feature(feature2.encode('utf-8')),
        'image_phash': _bytes_feature(feature3.encode('utf-8')),
        'title': _bytes_feature(feature4.encode('utf-8')),
        'label_group': _int64_feature(feature5)
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

df = pd.read_csv('/content/train.csv')
PATH = '/content/train_images/'
IMGS = list(df.image)
SIZE = 2048
CT = len(IMGS)//SIZE + int(len(IMGS)%SIZE!=0)

for j in tqdm(range(CT)):
    CT2 = min(SIZE, len(IMGS) - j*SIZE) 
    with tf.io.TFRecordWriter('./train_tfrec/train%.2i-%i.tfrec'%(j, CT2)) as writer: 
        for k in range(CT2): 
            img = cv2.imread(PATH+IMGS[j*SIZE + k])
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) # Fix incorrect colors
            img = cv2.imencode('.jpg', img, (cv2.IMWRITE_JPEG_QUALITY, 94))[1].tobytes()

            name = IMGS[SIZE*j+k]
            row = df.loc[df.image==name]
            example = serialize_example(
                row.posting_id.values[0],
                img,
                name,
                row.image_phash.values[0],
                row.title.values[0],
                row.label_group.values[0])
    
            writer.write(example)


100%|██████████| 17/17 [09:57<00:00, 35.16s/it]


## Load data

In [68]:
# Data access
GCS_PATH = '/content'
AUTO = tf.data.experimental.AUTOTUNE
# Configuration
EPOCHS = 15
BATCH_SIZE = 8 * strategy.num_replicas_in_sync
IMAGE_SIZE = [512, 512]
# Seed
SEED = 42
# Learning rate
LR = 0.001
# Verbosity
VERBOSE = 1
# Number of classe
N_CLASSES = 11014
# Number of folds
FOLDS = 5

# Training filenames directory
TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/*.tfrec')

In [55]:
# Function to get our f1 score
def f1_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    intersection = np.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)])
    len_y_pred = y_pred.apply(lambda x: len(x)).values
    len_y_true = y_true.apply(lambda x: len(x)).values
    f1 = 2 * intersection / (len_y_pred + len_y_true)
    return f1

# Function to seed everything
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
    
def arcface_format(posting_id, image, label_group, matches):
    return posting_id, {'inp1': image, 'inp2': label_group}, label_group, matches

# Data augmentation function
def data_augment(posting_id, image, label_group, matches):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_hue(image, 0.01)
    image = tf.image.random_saturation(image, 0.70, 1.30)
    image = tf.image.random_contrast(image, 0.80, 1.20)
    image = tf.image.random_brightness(image, 0.10)
    return posting_id, image, label_group, matches

# Function to decode our images
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels = 3)
    image = tf.image.resize(image, IMAGE_SIZE)
    image = tf.cast(image, tf.float32) / 255.0
    return image

# This function parse our images and also get the target variable
def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "posting_id": tf.io.FixedLenFeature([], tf.string),
        "image": tf.io.FixedLenFeature([], tf.string),
        "label_group": tf.io.FixedLenFeature([], tf.int64),
        "matches": tf.io.FixedLenFeature([], tf.string)
    }

    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    posting_id = example['posting_id']
    image = decode_image(example['image'])
#     label_group = tf.one_hot(tf.cast(example['label_group'], tf.int32), depth = N_CLASSES)
    label_group = tf.cast(example['label_group'], tf.int32)
    matches = example['matches']
    return posting_id, image, label_group, matches

# This function loads TF Records and parse them into tensors
def load_dataset(filenames, ordered = False):
    
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False 
        
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(read_labeled_tfrecord, num_parallel_calls = AUTO) 
    return dataset

# This function is to get our training tensors
def get_training_dataset(filenames, ordered = False):
    dataset = load_dataset(filenames, ordered = ordered)
    dataset = dataset.map(data_augment, num_parallel_calls = AUTO)
    dataset = dataset.map(arcface_format, num_parallel_calls = AUTO)
    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

# This function is to get our validation tensors
def get_validation_dataset(filenames, ordered = True):
    dataset = load_dataset(filenames, ordered = ordered)
    dataset = dataset.map(arcface_format, num_parallel_calls = AUTO)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) 
    return dataset

# Function to count how many photos we have in
def count_data_items(filenames):
    # The number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
print(f'Dataset: {NUM_TRAINING_IMAGES} training images')

Dataset: 34250 training images


# Image Model

## Build Model

In [84]:
# Function for a custom learning rate scheduler with warmup and decay
def get_lr_callback():
    lr_start   = 0.000001
    lr_max     = 0.000005 * BATCH_SIZE
    lr_min     = 0.000001
    lr_ramp_ep = 5
    lr_sus_ep  = 5
    lr_decay   = 0.8
   
    def lrfn(epoch):
        if epoch < 5:
            lr = lr_max    
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min    
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)
    return lr_callback

# Arcmarginproduct class keras layer
class ArcMarginProduct(tf.keras.layers.Layer):
    '''
    Implements large margin arc distance.

    Reference:
        https://arxiv.org/pdf/1801.07698.pdf
        https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/
            blob/master/src/modeling/metric_learning.py
    '''
    def __init__(self, n_classes, s=30, m=0.50, easy_margin=False,
                 ls_eps=0.0, **kwargs):

        super(ArcMarginProduct, self).__init__(**kwargs)

        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.ls_eps = ls_eps
        self.easy_margin = easy_margin
        self.cos_m = tf.math.cos(m)
        self.sin_m = tf.math.sin(m)
        self.th = tf.math.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'ls_eps': self.ls_eps,
            'easy_margin': self.easy_margin,
        })
        return config

    def build(self, input_shape):
        super(ArcMarginProduct, self).build(input_shape[0])

        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[0][-1]), self.n_classes),
            initializer='glorot_uniform',
            dtype='float32',
            trainable=True,
            regularizer=None)

    def call(self, inputs):
        X, y = inputs
        y = tf.cast(y, dtype=tf.int32)
        cosine = tf.matmul(
            tf.math.l2_normalize(X, axis=1),
            tf.math.l2_normalize(self.W, axis=0)
        )
        sine = tf.math.sqrt(1.0 - tf.math.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = tf.where(cosine > 0, phi, cosine)
        else:
            phi = tf.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = tf.cast(
            tf.one_hot(y, depth=self.n_classes),
            dtype=cosine.dtype
        )
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.n_classes

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output


# Function to create our EfficientNetB3 model
def get_model():

    with strategy.scope():

        margin = ArcMarginProduct(
            n_classes = N_CLASSES, 
            s = 30, 
            m = 0.7, 
            name='head/arc_margin', 
            dtype='float32'
            )

        inp = tf.keras.layers.Input(shape = (*IMAGE_SIZE, 3), name = 'inp1')
        label = tf.keras.layers.Input(shape = (), name = 'inp2')
        x = EfficientNetB3(weights = 'imagenet', include_top = False)(inp)
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = margin([x, label])
        
        output = tf.keras.layers.Softmax(dtype='float32')(x)

        model = tf.keras.models.Model(inputs = [inp, label], outputs = [output])

        opt = tf.keras.optimizers.Adam(learning_rate = LR)

        model.compile(
            optimizer = opt,
            loss = [tf.keras.losses.SparseCategoricalCrossentropy()],
            metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
            ) 
        
        return model

def build_model():
    margin = ArcMarginProduct(
            n_classes = N_CLASSES, 
            s = 30, 
            m = 0.7, 
            name='head/arc_margin', 
            dtype='float32'
            )

    inp = tf.keras.layers.Input(shape = (*IMAGE_SIZE, 3), name = 'inp1')
    label = tf.keras.layers.Input(shape = (), name = 'inp2')
    x = EfficientNetB3(weights = None, include_top = False)(inp)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = margin([x, label])
        
    output = tf.keras.layers.Softmax(dtype='float32')(x)

    model = tf.keras.models.Model(inputs = [inp, label], outputs = [output])
    model.load_weights('/content/EfficientNetB3_512_42.h5')

    opt = tf.keras.optimizers.Adam(learning_rate = LR)

    model.compile(
        optimizer = opt,
        loss = [tf.keras.losses.SparseCategoricalCrossentropy()],
        metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
        ) 

    return model 

## Train Model

In [85]:
# Seed everything
seed_everything(SEED)

print('\n')
print('-'*50)

train_dataset = get_training_dataset(TRAINING_FILENAMES, ordered = False)
train_dataset = train_dataset.map(lambda posting_id, image, label_group, matches: (image, label_group))

STEPS_PER_EPOCH = count_data_items(TRAINING_FILENAMES) // BATCH_SIZE
K.clear_session()
# model = get_model()
# Model checkpoint
checkpoint = tf.keras.callbacks.ModelCheckpoint(f'EfficientNetB3_{IMAGE_SIZE[0]}_{SEED}.h5', 
                                                monitor = 'loss', 
                                                verbose = VERBOSE, 
                                                save_best_only = True,
                                                save_weights_only = True, 
                                                mode = 'min')

model = build_model()

history = model.fit(train_dataset,
                    steps_per_epoch = STEPS_PER_EPOCH,
                    epochs = 10,
                    callbacks = [checkpoint, get_lr_callback()], 
                    verbose = VERBOSE)



--------------------------------------------------
Epoch 1/10

Epoch 00001: LearningRateScheduler reducing learning rate to 4e-05.
4281/4281 [==============================] - 2364s 547ms/step - loss: 21.4322 - sparse_categorical_accuracy: 0.0000e+00

Epoch 00001: loss improved from inf to 21.29479, saving model to EfficientNetB3_512_42.h5
Epoch 2/10

Epoch 00002: LearningRateScheduler reducing learning rate to 4e-05.
4281/4281 [==============================] - 2345s 548ms/step - loss: 20.2776 - sparse_categorical_accuracy: 0.0000e+00

Epoch 00002: loss improved from 21.29479 to 20.09633, saving model to EfficientNetB3_512_42.h5
Epoch 3/10

Epoch 00003: LearningRateScheduler reducing learning rate to 4e-05.
4281/4281 [==============================] - 2344s 547ms/step - loss: 19.6507 - sparse_categorical_accuracy: 0.0000e+00

Epoch 00003: loss improved from 20.09633 to 19.54295, saving model to EfficientNetB3_512_42.h5
Epoch 4/10

Epoch 00004: LearningRateScheduler reducing learning

In [86]:
!cp ./EfficientNetB3_512_42.h5 ./drive/MyDrive/data/